<a href="https://colab.research.google.com/github/WajdAlsuhaymi/IT326-DataMining-Group2/blob/main/Reports/Phase1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phase 1 – Data Selection

## Project Goal
The main goal of this project is to explore the relationship between social media engagement and emotional well-being. Through data analysis, the project aims to uncover how factors such as time spent, purpose of use, and users’ demographic characteristics influence psychological outcomes. The project seeks to raise awareness about the psychological effects of social media.

## Dataset Source
Kaggle: [Social Media Usage and Emotional Well-being](https://www.kaggle.com/datasets/emirhanai/social-media-usage-and-emotional-well-being)


Dataset Description and **Exploration**

In [9]:
import pandas as pd


url = "https://raw.githubusercontent.com/WajdAlSuhaymi/IT326-DataMining-Group2/main/Dataset/Raw_dataset.csv"

# قراءة الملف
df = pd.read_csv(url)

# عرض أول 5 صفوف للتأكد
df.head()


,User_ID,Age,Gender,Platform,Daily_Usage_Time (minutes),Posts_Per_Day,Likes_Received_Per_Day,Comments_Received_Per_Day,Messages_Sent_Per_Day,Dominant_Emotion
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,25,Female,Instagram,120.0,3.0,45.0,10.0,12.0,Happiness
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,30,Male,Twitter,90.0,5.0,20.0,25.0,30.0,Anger
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
import pandas as pd

# قراءة مجموعة البيانات
df = pd.read_csv('Raw_dataset.csv')

# عدد الصفوف والأعمدة
num_rows, num_columns = df.shape
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

# أنواع البيانات في كل عمود
print("\nData Types of Attributes (Features):")
print(df.dtypes)

# عرض بعض العينات من البيانات
print("\nSample of the dataset:")
print(df.head())

# عرض القيم المفقودة
print("\nMissing Values per Column:")
print(df.isnull().sum())

# الإحصائيات العامة للأعمدة الرقمية
print("\nStatistical Summary of Numerical Columns:")
print(df.describe())


Number of rows: 1001
Number of columns: 10

Data Types of Attributes (Features):
User_ID                        object
Age                            object
Gender                         object
Platform                       object
Daily_Usage_Time (minutes)    float64
Posts_Per_Day                 float64
Likes_Received_Per_Day        float64
Comments_Received_Per_Day     float64
Messages_Sent_Per_Day         float64
Dominant_Emotion               object
dtype: object

Sample of the dataset:
  User_ID Age      Gender   Platform  Daily_Usage_Time (minutes)  \
0       1  25      Female  Instagram                       120.0   
1       2  30        Male    Twitter                        90.0   
2       3  22  Non-binary   Facebook                        60.0   
3       4  28      Female  Instagram                       200.0   
4       5  33        Male   LinkedIn                        45.0   

   Posts_Per_Day  Likes_Received_Per_Day  Comments_Received_Per_Day  \
0            3.0     

In [8]:
import pandas as pd

# قراءة البيانات
df = pd.read_csv('Raw_dataset.csv', encoding='utf-8', on_bad_lines='skip')

# عدد الصفوف والأعمدة
print("Instances (rows), Features (cols):", df.shape)

# أنواع البيانات + القيم الفريدة + المفقودة
summary = pd.DataFrame({
    "dtype": df.dtypes.astype(str),
    "n_unique": df.nunique(),
    "n_missing": df.isna().sum()
}).sort_index()

display(summary)

print("\nTop columns with missing values:")
display(summary.sort_values("n_missing", ascending=False).head(10))

# عينة من البيانات
print("\nSample of the dataset:")
display(df.head())

# الإحصائيات الرقمية
print("\nStatistical Summary of Numerical Columns:")
display(df.describe())


Instances (rows), Features (cols): (1001, 10)


,dtype,n_unique,n_missing
Age,object,19,0
Comments_Received_Per_Day,float64,30,1
Daily_Usage_Time (minutes),float64,30,1
Dominant_Emotion,object,6,1
Gender,object,18,1
Likes_Received_Per_Day,float64,49,1
Messages_Sent_Per_Day,float64,29,1
Platform,object,7,1
Posts_Per_Day,float64,8,1
User_ID,object,1001,0



Top columns with missing values:


,dtype,n_unique,n_missing
Comments_Received_Per_Day,float64,30,1
Daily_Usage_Time (minutes),float64,30,1
Messages_Sent_Per_Day,float64,29,1
Dominant_Emotion,object,6,1
Gender,object,18,1
Likes_Received_Per_Day,float64,49,1
Posts_Per_Day,float64,8,1
Platform,object,7,1
Age,object,19,0
User_ID,object,1001,0



Sample of the dataset:


,User_ID,Age,Gender,Platform,Daily_Usage_Time (minutes),Posts_Per_Day,Likes_Received_Per_Day,Comments_Received_Per_Day,Messages_Sent_Per_Day,Dominant_Emotion
0,1,25,Female,Instagram,120.0,3.0,45.0,10.0,12.0,Happiness
1,2,30,Male,Twitter,90.0,5.0,20.0,25.0,30.0,Anger
2,3,22,Non-binary,Facebook,60.0,2.0,15.0,5.0,20.0,Neutral
3,4,28,Female,Instagram,200.0,8.0,100.0,30.0,50.0,Anxiety
4,5,33,Male,LinkedIn,45.0,1.0,5.0,2.0,10.0,Boredom



Statistical Summary of Numerical Columns:


,Daily_Usage_Time (minutes),Posts_Per_Day,Likes_Received_Per_Day,Comments_Received_Per_Day,Messages_Sent_Per_Day
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,95.950000,3.321000,39.898000,15.611000,22.560000
std,38.850442,1.914582,26.393867,8.819493,8.516274
min,40.000000,1.000000,5.000000,2.000000,8.000000
25%,65.000000,2.000000,20.000000,8.000000,17.750000
50%,85.000000,3.000000,33.000000,14.000000,22.000000
75%,120.000000,4.000000,55.000000,22.000000,28.000000
max,200.000000,8.000000,110.000000,40.000000,50.000000
